<a href="https://colab.research.google.com/github/Buddyalok/inventory_replenishment_model/blob/main/Inventory_Model_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [253]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [254]:
import numpy as np
import pandas as pd
from scipy.stats import norm

inv_snap = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Inv_Model_Project/inventory_snapshot.csv")
proc_snap = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Inv_Model_Project/procurement_data.csv")
sales = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Inv_Model_Project/sales_table.csv")
sku = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Inv_Model_Project/sku_table.csv")
vendor = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Inv_Model_Project/vendor_master.csv")
config_growth = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQJbh0k6AcI66Q_DZ3TbIb73z89iWAx-NIMpoAN2Ww-q-O17OnNL1nWYVGUfM5zCopHUN_9zJTPCa9U/pub?gid=1056909405&single=true&output=csv")
config_tat = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQJbh0k6AcI66Q_DZ3TbIb73z89iWAx-NIMpoAN2Ww-q-O17OnNL1nWYVGUfM5zCopHUN_9zJTPCa9U/pub?gid=206936273&single=true&output=csv")
service_level = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQJbh0k6AcI66Q_DZ3TbIb73z89iWAx-NIMpoAN2Ww-q-O17OnNL1nWYVGUfM5zCopHUN_9zJTPCa9U/pub?gid=1679244212&single=true&output=csv")
min_sales = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQJbh0k6AcI66Q_DZ3TbIb73z89iWAx-NIMpoAN2Ww-q-O17OnNL1nWYVGUfM5zCopHUN_9zJTPCa9U/pub?gid=753240523&single=true&output=csv")
moq_lotsize = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQJbh0k6AcI66Q_DZ3TbIb73z89iWAx-NIMpoAN2Ww-q-O17OnNL1nWYVGUfM5zCopHUN_9zJTPCa9U/pub?gid=694459612&single=true&output=csv")
# s_level.info()

In [255]:
# Calculating L50 stockout_days from 24th June 2025

inv_snap['inv_date'] = pd.to_datetime(inv_snap['snapshot_date']) # Convert 'inv_date' to datetime objects
end_date = pd.to_datetime('2025-06-24') # Define the end date and calculate the start date for the last 50 days
start_date = end_date - pd.Timedelta(days=49) # Including the end date makes it 50 days
inv_snap_filtered = inv_snap[(inv_snap['inv_date'] >= start_date) & (inv_snap['inv_date'] <= end_date)].copy() # Filter inv_snap for the last 50 days up to 2025-05-25
inv_snap_filtered['is_stockout'] = (inv_snap_filtered['Inventory Units'] == 0).astype(int) # Identify stockout days (where available_stock is 0)
stockout_days = inv_snap_filtered.groupby(['SKU_Code', 'Facilities'])['is_stockout'].sum().reset_index() # Group by sku_code and facility_id and sum the 'is_stockout' column
stockout_days.rename(columns={'is_stockout': 'stockout_days', 'SKU_Code': 'sku_code', 'Facilities': 'facility_code'}, inplace=True) # Rename the sum column to reflect stockout days

In [256]:
# Getting Sales data Ready
sales['sales_date'] = pd.to_datetime(sales['sales_date'], format='%d-%m-%Y') # Corecting date objct to datetime function
date_threshold = pd.to_datetime("2025-06-24") - pd.Timedelta(days=50) # calculating date before 50 days from 26th may
sales_filtered = sales.query('sales_date >= @date_threshold').copy() #filtering sales for 50 days
total_sales = sales_filtered.groupby(["sku_code", "facility_code"]).agg({ "total_qty":"sum"}) #aggregating sales
total_sales = pd.merge(total_sales, min_sales, on=['sku_code', 'facility_code'])  #merging min sales data into total sales to handle errors
mask_zero_negative_sales = total_sales['total_qty'] <= 0 # Identify rows with zero or negative total_sales
total_sales['total_qty'] = total_sales.groupby(['sku_code', 'facility_code'])['total_qty'].transform(lambda x: total_sales['min_sale_qty'][x.index] if any(x <= 0) else x) #overriding sales data whereever error is there
total_sales['total_qty'] = total_sales['total_qty'].round(2)
sales_stdev = sales_filtered.groupby(["sku_code", "facility_code"])["total_qty"].std()
sales_stdev_final = sales_stdev.reset_index(name='stdev_sales').round(2)
total_sales['pds_wo_oos'] = (total_sales['total_qty'] / 50).round(2)
total_sales = pd.merge(total_sales, stockout_days, on=['sku_code', 'facility_code'])
total_sales['pds_w_stockout'] = ((total_sales['pds_wo_oos'] / (50 - total_sales['pds_wo_oos'])) * 50).round(2)
total_sales = pd.merge(total_sales, sales_stdev_final, on=['sku_code', 'facility_code'])
# Projecting Sales from here----------
sales_final = pd.merge(total_sales, config_growth, on=['sku_code', 'facility_code'])
sales_final['n30_sales'] = 30 * (sales_final['pds_w_stockout'] * (1 + sales_final['growth'])).round(2)
sales_final['new_pds'] = (sales_final['n30_sales'] / 30).round(2)

In [257]:
curr_inv_filtered = inv_snap.query('snapshot_date == "2025-06-24"').copy() # Filtering Inventory Snapshot by date
curr_inv_filtered.rename(columns={"SKU_Code": "sku_code", "Facilities": "facility_code"}, inplace=True) # Renaming columns for easiness
curr_inv = curr_inv_filtered.groupby(["sku_code", "facility_code"])["Inventory Units"].sum() # Final Current_Inv Column
sit = curr_inv_filtered.groupby(["sku_code", "facility_code"])["inventory_in_transfer_qty"].sum() # Final SIT Column
in_transit = proc_snap.groupby(["sku_code", "facility_code"])["product_qty"].sum().copy() # Filtering In Transit Units (Assumuing proc_snap data is already filtered based on last 60 days data)

In [258]:
# Convert Series to DataFrames and reset index and rename
curr_inv_df = curr_inv.reset_index(name='inv')
sit_df = sit.reset_index(name='wh_transfer')
in_transit_df = in_transit.reset_index(name='po')

# Merge the DataFrames
df = pd.merge(curr_inv_df, sit_df, on=['sku_code', 'facility_code'], how='left')
df = pd.merge(df, in_transit_df, on=['sku_code', 'facility_code'], how='left').fillna(0)
df = pd.merge(df, config_tat, on=['sku_code', 'facility_code'], how='left').fillna(0)
df = pd.merge(df, service_level, on=['sku_code', 'facility_code'], how='left').fillna(0.95)
df['z_score'] = norm.ppf(df['ser_lvl']).round(3)
df = pd.merge(df, sales_final, on=['sku_code', 'facility_code'], how='left')
df['safety_qty']    = (df['z_score'] * np.sqrt(df['avg_tat'] * (df['stdev_sales'])**2 + (df['new_pds'] * df['stdev_tat'])**2 )).round(2)
df['safety_days']   = (df['safety_qty'] / df['new_pds']).round()
df['curr_doi']      = (df['inv'] / df['new_pds']).round()
df['reorder_point'] = (df['safety_days'] + df['avg_tat']).round()
df['post_grn_doi']  = ((df['inv'] + df['wh_transfer'] + df['po']) / df['new_pds'] ).round()
df['order_days'] = df.apply(lambda row: row['reorder_point'] - row['post_grn_doi'] if row['post_grn_doi'] < row['reorder_point'] else 0, axis=1).round(2)
df['order_units'] = (df['order_days'] * df['new_pds']).round()
df = pd.merge(df, moq_lotsize, on=['sku_code', 'facility_code'], how='left')


In [259]:
#Applying MOQ and LotSize into ordering qty to order according to the vendor or logistical requirements
import math

def calculate_final_order_qty(data):
    if data['order_units'] > 0:
        lot_count = math.ceil(data['order_units'] / data['lot_size'])
        initial_order_qty = lot_count * data['lot_size']
        return max(initial_order_qty, data['moq'])
    else:
        return 0

df['final_order_qty'] = df.apply(calculate_final_order_qty, axis=1)
inv_planning_model_final = df.query('order_days > 0').copy()
inv_planning_model_final

,sku_code,facility_code,inv,wh_transfer,po,avg_tat,stdev_tat,ser_lvl,z_score,total_qty,...,safety_qty,safety_days,curr_doi,reorder_point,post_grn_doi,order_days,order_units,moq,lot_size,final_order_qty
5,BGT_SKU_002,WH-GGN,0,0,0.0,11,5,0.999,3.09,48.0,...,23.01,20.0,0.0,31.0,0.0,31.0,37.0,50,5,50
12,BGT_SKU_004,WH-BLR,0,0,0.0,10,5,0.999,3.09,37.0,...,21.29,24.0,0.0,34.0,0.0,34.0,31.0,100,5,100
20,BGT_SKU_006,WH-BLR,0,0,0.0,11,3,0.999,3.09,26.0,...,15.34,24.0,0.0,35.0,0.0,35.0,22.0,100,5,100
21,BGT_SKU_006,WH-GGN,0,0,0.0,15,5,0.999,3.09,30.0,...,18.45,25.0,0.0,40.0,0.0,40.0,29.0,50,5,50
22,BGT_SKU_006,WH-KOL,0,0,0.0,10,4,0.999,3.09,38.0,...,18.63,20.0,0.0,30.0,0.0,30.0,28.0,100,5,100
72,BGT_SKU_019,WH-BLR,28,0,0.0,13,2,0.999,3.09,47.0,...,17.24,15.0,24.0,28.0,24.0,4.0,5.0,100,5,100
101,BGT_SKU_026,WH-GGN,0,0,0.0,12,7,0.999,3.09,47.0,...,27.90,24.0,0.0,36.0,0.0,36.0,41.0,50,5,50
129,BGT_SKU_033,WH-GGN,0,0,0.0,10,4,0.999,3.09,37.0,...,20.32,23.0,0.0,33.0,0.0,33.0,30.0,50,5,50
132,BGT_SKU_034,WH-BLR,0,0,0.0,11,6,0.999,3.09,53.0,...,28.42,22.0,0.0,33.0,0.0,33.0,43.0,100,5,100
140,BGT_SKU_036,WH-BLR,6,0,0.0,12,6,0.999,3.09,34.0,...,21.40,26.0,7.0,38.0,7.0,31.0,26.0,100,5,100
